## **Alineamiento de los versículos en ambos idiomas**

### Librerías a usar

In [1]:
import os
import json
import csv

### Alineación de los versículos en Aymara con los versículos en Español

In [2]:
raw_aymara_bible_paths = ["data/raw/bible/aymara/293", "data/raw/bible/aymara/2250"]
raw_spanish_bible_paths = ["data/raw/bible/spanish/1782", "data/raw/bible/spanish/4278"]

aymara_all = []
spanish_all = []

for raw_aymara_bible_path in raw_aymara_bible_paths:
    for raw_spanish_bible_path in raw_spanish_bible_paths:

        print(f"Alineando: {raw_aymara_bible_path} ↔ {raw_spanish_bible_path}")

        aymara_dict = {}
        spanish_dict = {}

        # Cargamos Biblia en Aymara
        for filename in os.listdir(raw_aymara_bible_path):
            if filename.endswith(".json"):
                with open(os.path.join(raw_aymara_bible_path, filename), "r", encoding="utf-8") as f:
                    aymara_dict.update(json.load(f))

        # Cargamos Biblia en Español
        for filename in os.listdir(raw_spanish_bible_path):
            if filename.endswith(".json"):
                with open(os.path.join(raw_spanish_bible_path, filename), "r", encoding="utf-8") as f:
                    spanish_dict.update(json.load(f))

        # Alineamos versículos
        for verse_id in aymara_dict:
            if verse_id in spanish_dict:
                ay = aymara_dict[verse_id].strip()
                es = spanish_dict[verse_id].strip()

                if ay and es:
                    aymara_all.append(ay)
                    spanish_all.append(es)

print()
print(f"Total versículos en Aymara: {len(aymara_all)}")
print(f"Total versículos en Español: {len(spanish_all)}")


Alineando: data/raw/bible/aymara/293 ↔ data/raw/bible/spanish/1782
Alineando: data/raw/bible/aymara/293 ↔ data/raw/bible/spanish/4278
Alineando: data/raw/bible/aymara/2250 ↔ data/raw/bible/spanish/1782
Alineando: data/raw/bible/aymara/2250 ↔ data/raw/bible/spanish/4278

Total versículos en Aymara: 122140
Total versículos en Español: 122140


## **Preprocesamiento del corpus**

### Librerías a usar

In [3]:
import pandas as pd
import unicodedata
import re
import csv
import os

### Visualizamos los caracteres que conforman los textos

In [4]:
# Caracteres únicos en Español
spanish_chars = sorted(set("".join([verse.lower() for verse in spanish_all])))
print(spanish_chars)
print("Cantidad de caracteres en español:", len(spanish_chars))

[' ', '!', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', '«', '»', '¿', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü', '—', '‘', '’', '“', '”', '…']
Cantidad de caracteres en español: 64


In [5]:
# Caracteres únicos en Aymara
aymara_chars = sorted(set("".join([verse.lower() for verse in aymara_all])))
print(aymara_chars)
print("Cantidad de caracteres en aymara:", len(aymara_chars))

[' ', '!', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '¿', 'á', 'ä', 'é', 'ë', 'í', 'ï', 'ñ', 'ó', 'ö', 'ú', 'ü', '—', '‘', '’', '“', '”']
Cantidad de caracteres en aymara: 67


### Pipeline de preprocesamiento

In [6]:
def normalize_unicode(text: str) -> str:
    '''Para asegurarnos de que los caracteres se representen de una forma estándar interna'''
    return unicodedata.normalize("NFC", text)

def normalize_quotes(text: str) -> str:
    '''Para normalizar las comillas a comillas simples estándar ASCII'''
    text = text.replace("«", "\"").replace("»", "\"")
    text = text.replace("“", "\"").replace("”", "\"")
    text = text.replace("‘", "'").replace("’", "'")
    return text

def normalize_punctuation(text: str) -> str:
    '''Para normalizar signos de puntuación especiales'''
    text = text.replace("—", "-")
    text = text.replace("…", "...")
    return text

def clean_brackets(text: str) -> str:
    '''Para eliminar los corchetes [ y ] del texto'''
    return text.replace("[", "").replace("]", "")

def remove_verse_numbers(text: str) -> str:
    '''Cualquier número en el texto hace referencia a un código de versículo'''
    words = text.split()
    cleaned_words = [w for w in words if not any(ch.isdigit() for ch in w)] # Mantenemos solo palabras que no contienen dígitos
    return " ".join(cleaned_words)

def clean_spaces(text: str) -> str:
    '''Para normalizar los espacios en el texto'''
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def lowercase(text: str) -> str:
    '''Para convertir todo el texto a minúsculas'''
    return text.lower()

def preprocess(text: str) -> str:
    text = normalize_unicode(text)
    text = normalize_quotes(text)
    text = normalize_punctuation(text)
    text = clean_brackets(text)
    text = remove_verse_numbers(text)
    text = clean_spaces(text)
    text = lowercase(text)
    return text

### Aplicamos preprocesamiento

In [7]:
spanish_lines = []
aymara_lines = []

for spanish_raw, aymara_raw in zip(spanish_all, aymara_all):

    spanish_clean = preprocess(spanish_raw)
    aymara_clean = preprocess(aymara_raw)

    if spanish_clean and aymara_clean:
        spanish_lines.append(spanish_clean)
        aymara_lines.append(aymara_clean)

print(f"Cantidad de versículos en español: {len(spanish_lines)}")
print(f"Cantidad de versículos en aymara: {len(aymara_lines)}")

Cantidad de versículos en español: 122140
Cantidad de versículos en aymara: 122140


## **Dividimos en oraciones**

In [8]:
def split_by_dot(text):
    """
    Función para dividir el texto por punto (.) mantiendo el punto en cada oración.
    """
    text = text.strip()

    # Dividimos después del punto + espacios
    sentences = re.split(r'(?<=\.)\s+', text)

    # Limpieza de cada oración
    sentences = [s.strip() for s in sentences if s.strip()]

    return sentences

In [9]:
spanish_texts = []
aymara_texts = []

for es, ay in zip(spanish_lines, aymara_lines):
    es = es.strip()
    ay = ay.strip()

    if not es or not ay:
        continue

    # Guardamos siempre los versículos completos
    spanish_texts.append(es)
    aymara_texts.append(ay)

    # Dividimos versículos por punto
    es_sents = split_by_dot(es)
    ay_sents = split_by_dot(ay)

    # Guardamos oraciones solo si hay alineación perfecta
    if len(es_sents) == len(ay_sents) and len(es_sents) > 1:
        for es_sent, ay_sent in zip(es_sents, ay_sents):
            # Filtramos ruido
            if len(es_sent) > 5 and len(ay_sent) > 5:
                spanish_texts.append(es_sent)
                aymara_texts.append(ay_sent)

print(f"Cantidad de textos en español: {len(spanish_texts)}")
print(f"Cantidad de textos en aymara: {len(aymara_texts)}")

Cantidad de textos en español: 162671
Cantidad de textos en aymara: 162671


In [10]:
# Guardamos los resultados
folder_clean = "data/clean/bible"
os.makedirs(folder_clean, exist_ok=True)

with open(f"{folder_clean}/spanish.txt", "w", encoding="utf-8") as f:
    for line in spanish_texts:
        f.write(line + "\n")

with open(f"{folder_clean}/aymara.txt", "w", encoding="utf-8") as f:
    for line in aymara_texts:
        f.write(line + "\n")

print(f"Archivos {folder_clean}/spanish.txt y {folder_clean}/aymara.txt guardados exitosamente...")

Archivos data/clean/bible/spanish.txt y data/clean/bible/aymara.txt guardados exitosamente...
